In [ ]:
!apt-get update
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import numpy as np
import pandas as pd
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver', options=options)
songList = pd.read_csv(
    "https://raw.githubusercontent.com/JunHCha/K-POP-Lyrics-1964-2020/main/data/lyrics_original/kpop_lyrics_crawling_step1.csv", index_col=False
)

### STEP 2: Song_id에 해당하는 노래의 가사 추출 후, STEP 1의 데이터와 병합

In [ ]:
song_details = pd.DataFrame(columns=['lyric', 'genre'])
detail_base_url = "http://www.melon.com/song/detail.htm?songId="
lyric_tag_regex = re.compile("<.*?>")
song_ids = songList['song_id'].astype(str)

In [ ]:
for song_id in song_ids:

    wd.get(detail_base_url + song_id)

    try:
        lyric = wd.find_element(By.CSS_SELECTOR, "div.lyric").text
    except NoSuchElementException:
        lyric = ""
    print("-----------------------------------")
    print(song_id)
    print(lyric)
    try:
        genre = (
            wd.find_element(By.CSS_SELECTOR, "div.meta")
            .find_elements(By.CSS_SELECTOR, "dd")[2]
            .text
        )
    except NoSuchElementException:
        pass

    current = pd.DataFrame(
        {
            "song_id": [song_id],
            "lyric": [lyric],
            "genre": [genre],
        }
    )
    song_details = pd.concat((song_details, current))
    time.sleep(np.random.randint(1, 15))

In [ ]:
song_details["song_id"] = pd.to_numeric(song_details["song_id"])
song_list = pd.merge(
    songList, song_details, left_on="song_id", right_on="song_id", how="inner"
)
song_list.to_csv(
    "kpop_lyrics_crawling_step2.csv",
    encoding="utf-8",
    index=False,
)

In [ ]:
from google.colab import files
files.download("kpop_lyrics_crawling_step2.csv")